In [6]:
import os
import cv2
import numpy as np
np.random.seed(1337)
import tensorflow
from tensorflow import keras
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import Model, Sequential
import tensorflow.keras.layers
from tensorflow.keras.layers import Dropout, Dense, Flatten
from tensorflow.keras.layers import ConvLSTM2D
from sklearn.model_selection import train_test_split
import tensorflow.keras.metrics as km
from sklearn.metrics import classification_report
'''from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score'''

'from sklearn.metrics import accuracy_score\nfrom sklearn.metrics import precision_score\nfrom sklearn.metrics import recall_score\nfrom sklearn.metrics import f1_score\nfrom sklearn.metrics import cohen_kappa_score\nfrom sklearn.metrics import roc_auc_score'

In [17]:
data_dir = "D:/Real Life Violence Dataset"
img_height , img_width = 35,35
seq_len = 40
classes = ["Burglary","Normal","Theft","Non violence", "Violence"]

In [16]:
def frames_extraction(video_path):
    frames_list = []
     
    vidObj = cv2.VideoCapture(video_path)
    # Used as counter variable 
    count = 1
 
    while count <= seq_len: 
        success, image = vidObj.read() 
        if success:
            image = cv2.resize(image, (img_height, img_width))
            frames_list.append(image)
            count += 1
        else:
            print("Defected frame")
            break
 
            
    return frames_list

In [17]:
def create_data(input_dir):
    X = []
    Y = []
     
    classes_list = os.listdir(input_dir)
     
    for c in classes_list:
        print(c)
        files_list = os.listdir(os.path.join(input_dir, c))
        for f in files_list[:550]:
            frames = frames_extraction(os.path.join(os.path.join(input_dir, c), f))
            if len(frames) == seq_len:
                X.append(frames)
             
                y = [0]*len(classes)
                y[classes.index(c)] = 1
                Y.append(y)
     
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y
 
X, Y = create_data(data_dir)

Burglary
Non violence
Defected frame
Normal
Theft
Violence


In [24]:
print(Y[540:560])

[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 1 0]]


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=1)

In [26]:
model = Sequential()
model.add(ConvLSTM2D(filters = 72, kernel_size = (3, 3), return_sequences = False, data_format = "channels_last", input_shape = (seq_len, img_height, img_width, 3)))
#dataformat in the above line represents the rgb of image i.e. '3' dimensions will be given in the last 
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(254, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(5, activation = "softmax"))

In [27]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_3 (ConvLSTM2D)  (None, 33, 33, 72)        194688    
_________________________________________________________________
dropout_6 (Dropout)          (None, 33, 33, 72)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 78408)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 254)               19915886  
_________________________________________________________________
dropout_7 (Dropout)          (None, 254)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 1275      
Total params: 20,111,849
Trainable params: 20,111,849
Non-trainable params: 0
__________________________________________

In [9]:
opt = SGD(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [12]:
model.fit(x=X_train, y=y_train, epochs=21, batch_size = 3,shuffle = False, validation_split=0.2)

Train on 1759 samples, validate on 440 samples
Epoch 1/21
1759/1759 [==============================] - 601s 342ms/sample - loss: 1.2383 - accuracy: 0.4832 - val_loss: 0.9533 - val_accuracy: 0.6000
Epoch 2/21
1759/1759 [==============================] - 592s 336ms/sample - loss: 0.7705 - accuracy: 0.6936 - val_loss: 0.6607 - val_accuracy: 0.7364
Epoch 3/21
1759/1759 [==============================] - 591s 336ms/sample - loss: 0.5423 - accuracy: 0.8016 - val_loss: 0.5529 - val_accuracy: 0.7818
Epoch 4/21
1759/1759 [==============================] - 592s 336ms/sample - loss: 0.4454 - accuracy: 0.8380 - val_loss: 0.5262 - val_accuracy: 0.7864
Epoch 5/21
1759/1759 [==============================] - 592s 336ms/sample - loss: 0.3378 - accuracy: 0.8783 - val_loss: 0.4620 - val_accuracy: 0.8136
Epoch 6/21
1759/1759 [==============================] - 592s 336ms/sample - loss: 0.3132 - accuracy: 0.8920 - val_loss: 0.4359 - val_accuracy: 0.8136
Epoch 7/21
1759/1759 [==============================]

In [15]:
model.save('mymodelmain1')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: mymodelmain1\assets


In [16]:
y_pred = model.predict(X_test)

ResourceExhaustedError:  OOM when allocating tensor with shape[40,32,33,33,72] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential/conv_lst_m2d/TensorArrayV2Stack/TensorListStack (defined at <ipython-input-16-33f45f2977c2>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_distributed_function_55408]

Function call stack:
distributed_function


In [ ]:
y_pred = np.argmax(y_pred, axis = 1)
y_test = np.argmax(y_test, axis = 1)

In [ ]:
print(classification_report(y_test, y_pred))